In [ ]:
#install packages 
!pip install pandas --upgrade
!pip install tweepy --upgrade

     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from twitter_authentication import bearer_token
import tweepy
# For sending GET requests from the API
# For displaying the data after
import pandas as pd
# For parsing the dates received from twitter in readable formats
#To add wait time between requests
import time




In [ ]:
#%%
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)
climate_tweets = []
for response in tweepy.Paginator(client.search_all_tweets,
                                 query = 'Climate Change -is:retweet lang:en',
                                 user_fields = ['username', 'public_metrics', 'description', 'location','created_at'],
                                 tweet_fields = ['created_at','id','text','author_id','in_reply_to_user_id','geo','conversation_id','lang','public_metrics','referenced_tweets','reply_settings','source'],
                                 expansions = ['author_id,in_reply_to_user_id,geo.place_id'],
                                 start_time = "2021-12-30T00:00:00Z",
                                 end_time = "2021-12-31T00:00:00Z",
                                 max_results=10):
    time.sleep(1)






Rate limit exceeded. Sleeping for 409 seconds.


KeyboardInterrupt: ignored

In [ ]:
    climate_tweets.append(response)

    climate_tweets[0].data[0]

    climate_tweets[0].includes['users'][2]

    climate_tweets[0].includes['users'][2].description

    result = []
user_dict = {}
# Loop through each response object
for response in climate_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
  for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username,
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                              }
  for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id,
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                       })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

df

df.to_csv("output.csv", index=False)

#%%
